In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load the dataset
file_path = "everyday_2024_w_metro_station.csv"
df = pd.read_csv(file_path)

# Display first few rows to inspect data
print(df.head())

   Station Name        Date    Month  Season  Entries (AM Peak (Open-9:30am))  \
0  Addison Road  2024-01-01  January  Winter                              100   
1  Addison Road  2024-01-02  January  Winter                              642   
2  Addison Road  2024-01-03  January  Winter                              864   
3  Addison Road  2024-01-04  January  Winter                              858   
4  Addison Road  2024-01-05  January  Winter                              722   

   Entries (Evening (7pm-12am))  Entries (Late Night (12am-Close))  \
0                            66                                  2   
1                            96                                 15   
2                            84                                 10   
3                            76                                 31   
4                            82                                  1   

   Entries (Midday (9:30am-3pm))  Entries (PM Peak (3pm-7pm))  \
0                          

C:\Users\mayap\AppData\Local\Temp\ipykernel_25984\3937679489.py:7: DtypeWarning: Columns (24,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [2]:
# Define time period columns
traffic_columns = [
    "Entries (AM Peak (Open-9:30am))",
    "Entries (Midday (9:30am-3pm))",
    "Entries (PM Peak (3pm-7pm))",
    "Entries (Evening (7pm-12am))",
    "Entries (Late Night (12am-Close))"
]

# Convert entry columns to numeric (fixes errors where numbers are stored as text)
for col in traffic_columns:
    df[col] = pd.to_numeric(df[col], errors="coerce")
    
# Display summary statistics to check conversion
df[traffic_columns].describe()

,Entries (AM Peak (Open-9:30am)),Entries (Midday (9:30am-3pm)),Entries (PM Peak (3pm-7pm)),Entries (Evening (7pm-12am)),Entries (Late Night (12am-Close))
count,35752.000000,35752.000000,35752.000000,35752.000000,35752.000000
mean,871.777691,925.354665,1268.467666,499.832960,18.092722
std,840.445564,740.037515,1592.280650,806.400358,44.060204
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,218.000000,408.000000,327.000000,119.000000,3.000000
50%,646.000000,703.000000,638.000000,229.000000,8.000000
75%,1298.000000,1215.000000,1505.000000,560.000000,16.000000
max,6596.000000,19233.000000,19275.000000,16405.000000,2122.000000


In [3]:
# Compute total foot traffic per station per day
df["Total_Foot_Traffic"] = df[traffic_columns].sum(axis=1)

# Drop stations where Total_Foot_Traffic is 0 (closed stations)
df = df[df["Total_Foot_Traffic"] > 0]

# Display first few rows after filtering
df.head()

,Station Name,Date,Month,Season,Entries (AM Peak (Open-9:30am)),Entries (Evening (7pm-12am)),Entries (Late Night (12am-Close)),Entries (Midday (9:30am-3pm)),Entries (PM Peak (3pm-7pm)),Exits (AM Peak (Open-9:30am)),...,ADDRESS,NUMBER_OF_LINES,STATION_TYPE_1,STATION_TYPE_2,LINE_1,LINE_2,LINE_3,LINE_4,LINE_5,Total_Foot_Traffic
0,Addison Road,2024-01-01,January,Winter,100,66,2,135,95,45,...,"100 ADDISON ROAD S., CAPITOL HEIGHTS, MD",2,Residential,NaN,blue,silver,NaN,NaN,NaN,398
1,Addison Road,2024-01-02,January,Winter,642,96,15,322,148,114,...,"100 ADDISON ROAD S., CAPITOL HEIGHTS, MD",2,Residential,NaN,blue,silver,NaN,NaN,NaN,1223
2,Addison Road,2024-01-03,January,Winter,864,84,10,348,162,167,...,"100 ADDISON ROAD S., CAPITOL HEIGHTS, MD",2,Residential,NaN,blue,silver,NaN,NaN,NaN,1468
3,Addison Road,2024-01-04,January,Winter,858,76,31,313,172,173,...,"100 ADDISON ROAD S., CAPITOL HEIGHTS, MD",2,Residential,NaN,blue,silver,NaN,NaN,NaN,1450
4,Addison Road,2024-01-05,January,Winter,722,82,1,357,235,140,...,"100 ADDISON ROAD S., CAPITOL HEIGHTS, MD",2,Residential,NaN,blue,silver,NaN,NaN,NaN,1397


In [4]:
# Reshape the dataset so each row represents a specific station, date, and time period
df_melted = df.melt(
    id_vars=["Station Name", "Date"], 
    value_vars=traffic_columns, 
    var_name="Time Period", 
    value_name="Foot_Traffic"
)

# Display first few rows to confirm restructuring
df_melted.head()


,Station Name,Date,Time Period,Foot_Traffic
0,Addison Road,2024-01-01,Entries (AM Peak (Open-9:30am)),100
1,Addison Road,2024-01-02,Entries (AM Peak (Open-9:30am)),642
2,Addison Road,2024-01-03,Entries (AM Peak (Open-9:30am)),864
3,Addison Road,2024-01-04,Entries (AM Peak (Open-9:30am)),858
4,Addison Road,2024-01-05,Entries (AM Peak (Open-9:30am)),722


In [5]:
# Compute rolling average per station and time period (to smooth fluctuations)
df_melted["Station_Avg"] = (
    df_melted.groupby(["Station Name", "Time Period"])["Foot_Traffic"]
    .transform(lambda x: x.rolling(window=7, min_periods=1).mean())
)
# Switched from .rolling

# Display a sample of station averages
df_melted[["Station Name", "Time Period", "Station_Avg"]].drop_duplicates().head(20)

,Station Name,Time Period,Station_Avg
0,Addison Road,Entries (AM Peak (Open-9:30am)),100.000000
1,Addison Road,Entries (AM Peak (Open-9:30am)),371.000000
2,Addison Road,Entries (AM Peak (Open-9:30am)),535.333333
3,Addison Road,Entries (AM Peak (Open-9:30am)),616.000000
4,Addison Road,Entries (AM Peak (Open-9:30am)),637.200000
5,Addison Road,Entries (AM Peak (Open-9:30am)),553.000000
6,Addison Road,Entries (AM Peak (Open-9:30am)),487.714286
7,Addison Road,Entries (AM Peak (Open-9:30am)),590.857143
8,Addison Road,Entries (AM Peak (Open-9:30am)),627.428571
9,Addison Road,Entries (AM Peak (Open-9:30am)),632.428571


In [6]:
# Ensure no division errors
df_melted["Station_Avg"] = df_melted["Station_Avg"].replace(0, np.nan)  # Avoid division by zero

# Compute Dynamic Pricing Percentage with a safeguard against excessive negative values
df_melted["Dynamic_Price_Percentage"] = ((df_melted["Foot_Traffic"] - df_melted["Station_Avg"]) / df_melted["Station_Avg"]) * 100

# Ensure minimum value is -100% (foot traffic should never be less than 0)
df_melted["Dynamic_Price_Percentage"] = df_melted["Dynamic_Price_Percentage"].clip(lower=-100)

# Use percentile-based scaling instead of a hard cap
upper_limit = df_melted["Dynamic_Price_Percentage"].quantile(0.99)  # 99th percentile as cap
lower_limit = df_melted["Dynamic_Price_Percentage"].quantile(0.01)  # 1st percentile as min

# Clip based on real data distribution
df_melted["Dynamic_Price_Percentage"] = df_melted["Dynamic_Price_Percentage"].clip(lower=lower_limit, upper=upper_limit)

# Display first few rows to check calculation
df_melted[["Station Name", "Date", "Time Period", "Foot_Traffic", "Station_Avg", "Dynamic_Price_Percentage"]].head(20)


,Station Name,Date,Time Period,Foot_Traffic,Station_Avg,Dynamic_Price_Percentage
0,Addison Road,2024-01-01,Entries (AM Peak (Open-9:30am)),100,100.000000,0.000000
1,Addison Road,2024-01-02,Entries (AM Peak (Open-9:30am)),642,371.000000,73.045822
2,Addison Road,2024-01-03,Entries (AM Peak (Open-9:30am)),864,535.333333,61.394770
3,Addison Road,2024-01-04,Entries (AM Peak (Open-9:30am)),858,616.000000,39.285714
4,Addison Road,2024-01-05,Entries (AM Peak (Open-9:30am)),722,637.200000,13.308223
5,Addison Road,2024-01-06,Entries (AM Peak (Open-9:30am)),132,553.000000,-76.130199
6,Addison Road,2024-01-07,Entries (AM Peak (Open-9:30am)),96,487.714286,-80.316344
7,Addison Road,2024-01-08,Entries (AM Peak (Open-9:30am)),822,590.857143,39.119923
8,Addison Road,2024-01-09,Entries (AM Peak (Open-9:30am)),898,627.428571,43.123862
9,Addison Road,2024-01-10,Entries (AM Peak (Open-9:30am)),899,632.428571,42.150440


In [7]:
# 🔹 Sort by foot traffic (most crowded times appear first)
df_sorted = df_melted.sort_values(by="Foot_Traffic", ascending=False)

# Display top 20 busiest station-time slots with adjusted pricing
df_sorted[["Station Name", "Date", "Time Period", "Foot_Traffic", "Dynamic_Price_Percentage"]].head(10)

,Station Name,Date,Time Period,Foot_Traffic,Dynamic_Price_Percentage
99641,Smithsonian,2024-03-24,Entries (PM Peak (3pm-7pm)),19275,112.771829
62743,Rosslyn,2024-10-27,Entries (Midday (9:30am-3pm)),19233,184.798636
99647,Smithsonian,2024-03-30,Entries (PM Peak (3pm-7pm)),16869,52.579757
135483,Smithsonian,2024-07-04,Entries (Evening (7pm-12am)),16405,184.798636
129365,Navy Yard-Ballpark,2024-07-19,Entries (Evening (7pm-12am)),16222,174.616687
129323,Navy Yard-Ballpark,2024-06-07,Entries (Evening (7pm-12am)),15788,117.350089
129359,Navy Yard-Ballpark,2024-07-13,Entries (Evening (7pm-12am)),15085,184.798636
129366,Navy Yard-Ballpark,2024-07-20,Entries (Evening (7pm-12am)),14450,148.434238
125692,L'Enfant Plaza,2024-07-04,Entries (Evening (7pm-12am)),13448,184.798636
129349,Navy Yard-Ballpark,2024-07-03,Entries (Evening (7pm-12am)),13359,137.886034
